# Monte Carlo Dispersion Analysis with the Dispersion Class

Finally the Monte Carlo Simulations can be performed using a dedicated class called Dispersion. This class is a wrapper for the Monte Carlo Simulations, and it is the recommended way to perform the simulations. Say goodbye to the long and tedious process of creating the Monte Carlo Simulations throughout jupyter notebooks!

In [ ]:
%load_ext autoreload
%autoreload 2

First, let's import the necessary libraries, including the newest Dispersion class!

In [ ]:
from rocketpy import Environment, Rocket, SolidMotor, Flight, Dispersion


If you are using Jupyter Notebooks, it is recommended to run the following line to make matplotlib plots which will be shown later interactive and higher quality.

In [ ]:
%matplotlib inline

The Dispersion class allows us to perform Monte Carlo Simulations in a very simple way.
We just need to create an instance of the class, and then call the method run() to perform the simulations.
The class has a lot of capabilities, but we will only use a few of them in this example.
We encourage you to check the documentation of the class to learn more about the Dispersion.

Also, you can check RocketPy's main reference for a better conceptual understanding 
of the Monte Carlo Simulations: [RocketPy: Six Degree-of-Freedom Rocket Trajectory Simulator](https://doi.org/10.1061/(ASCE)AS.1943-5525.0001331)

We will describe two different options of usage:
- Using a Flight object as input, speeding up the process of creating the simulations. (currently not completely described in this notebook)
- Using a dictionary as input, including mean values and uncertainties for each parameter 

You need only one of them to get started. 

## 1st Option -> Use your Flight object as input for the Dispersion class

### Creating an Environment for 'Ponte de Sôr', Portugal

In [ ]:
Env = Environment(
    railLength=5.2, latitude=39.389700, longitude=-8.288964, elevation=113
)


To get weather data from the GFS forecast, available online, we run the following lines.

First, we set tomorrow's date.

In [ ]:
import datetime

tomorrow = datetime.date.today() + datetime.timedelta(days=1)

Env.setDate((tomorrow.year, tomorrow.month, tomorrow.day, 12))  # Hour given in UTC time


Then, we tell Env to use a GFS forecast to get the atmospheric conditions for flight.

Don't mind the warning, it just means that not all variables, such as wind speed or atmospheric temperature, are available at all altitudes given by the forecast.

In [ ]:
Env.setAtmosphericModel(type="Forecast", file="GFS")


In [ ]:
Env.info()


### Creating a Motor for the Rocket

We define a motor for the rocket, using the data from the manufacturer, and following
the [RocketPy's documentation](https://docs.rocketpy.org/en/latest/user/index.html).

In [ ]:
Pro75M1670 = SolidMotor(
    thrustSource="../../../data/motors/Cesaroni_M1670.eng",
    burnOut=3.9,
    grainsCenterOfMassPosition=-0.85704,
    grainNumber=5,
    grainSeparation=5 / 1000,
    grainDensity=1815,
    grainOuterRadius=33 / 1000,
    grainInitialInnerRadius=15 / 1000,
    grainInitialHeight=120 / 1000,
    nozzleRadius=33 / 1000,
    throatRadius=11 / 1000,
    interpolationMethod="linear",
    nozzlePosition=-1.255,
)


In [ ]:
Pro75M1670.info()


### Creating a Rocket

In [ ]:
Calisto = Rocket(
    radius=127 / 2000,
    mass=19.197 - 2.956,
    inertiaI=6.60,
    inertiaZ=0.0351,
    powerOffDrag="../../../data/calisto/powerOffDragCurve.csv",
    powerOnDrag="../../../data/calisto/powerOnDragCurve.csv",
    centerOfDryMassPosition=0,
    coordinateSystemOrientation="tailToNose",
)

Calisto.setRailButtons([0.2, -0.5])

Calisto.addMotor(Pro75M1670, position=-1.255)

NoseCone = Calisto.addNose(length=0.55829, kind="vonKarman", position=0.71971 + 0.55829)

FinSet = Calisto.addTrapezoidalFins(
    n=4,
    rootChord=0.120,
    tipChord=0.040,
    span=0.100,
    position=-1.04956,
    cantAngle=0,
    radius=None,
    airfoil=None,
)

Tail = Calisto.addTail(
    topRadius=0.0635, bottomRadius=0.0435, length=0.060, position=-1.194656
)



Additionally, we set parachutes for our Rocket, as well as the trigger functions for the deployment of such parachutes.

In [ ]:
def drogueTrigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate drogue when vz < 0 m/s.
    return True if y[5] < 0 else False


def mainTrigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate main when vz < 0 m/s and z < 500 + 100 m (+100 due to surface elevation).
    return True if y[5] < 0 and y[2] < 500 + 100 else False


Main = Calisto.addParachute(
    "Main",
    CdS=10.0,
    trigger=mainTrigger,
    samplingRate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

Drogue = Calisto.addParachute(
    "Drogue",
    CdS=1.0,
    trigger=drogueTrigger,
    samplingRate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)


In [ ]:
Calisto.allInfo()


### Simulate single flight

In [ ]:
TestFlight = Flight(
    rocket=Calisto,
    environment=Env,
    inclination=84,
    heading=133,
)


And we can visualize the flight trajectory:

In [ ]:
TestFlight.plots.trajectory_3d()


### Starting the Monte Carlo Simulations

First, let's invoke the Dispersion class, we only need a filename to initialize it.
The filename will be used either to save the results of the simulations or to load them
from a previous ran simulation.

In [ ]:
TestDispersion = Dispersion(filename="dispersion_analysis_outputs/disp_class_example")


Then, we can run the simulations using the method Dispersion.run_dispersion().
But before that, we need to set some simple parameters for the simulations.
We will set them by using a dictionary, which is one of the simplest way to do it.

In [ ]:
#TODO: explain that this is std only
disp_dictionary = {
    # Solid Motor Parameters
    "burnOutTime": 0.2,
    "totalImpulse": 0.033 * Pro75M1670.totalImpulse,
    "motor_position": (-1.255,0),
    # Rocket Parameters
    "mass": 0.100,
    "radius": 0.001,
    "powerOffDrag": 0.033,  # Multiplier
    "powerOnDrag": 0.033,  # Multiplier
    "parachute_Main_CdS": 1,
    "parachute_Drogue_CdS": 0.1,
    "parachute_Main_lag": 0.1,
    "parachute_Drogue_lag": 0.1,
    # Flight Parameters
    "inclination": 1,
    "heading": 2,
}


Finally, let's iterate over the simulations and export the data from each flight simulation!

In [ ]:
TestDispersion.run_dispersion(
    number_of_simulations=50,
    dispersion_dictionary=disp_dictionary,
    flight=TestFlight,
    append=False,
)


### Visualizing the results

Now we finally have the results of our Monte Carlo simulations loaded!
Let's play with them.

First, we can print numerical information regarding the results of the simulations.

In [ ]:
TestDispersion.import_results()


In [ ]:
TestDispersion.print_results()


Also, we can visualize histograms of such results

In [ ]:
TestDispersion.allInfo()


Export to kml so it can be visualized in Google Earth

In [ ]:
TestDispersion.exportEllipsesToKML(
    filename="dispersion_analysis_outputs/disp_class_example.kml",
    origin_lat=Env.latitude,
    origin_lon=Env.longitude,
    type="impact",
)


## 2nd Option -> Running by using only a dictionary of parameters

This second option allow us to perform the Monte Carlo Simulations without the need of a Flight object. This is useful when we want to perform the simulations for a rocket that we don't have a Flight object for, or when we want to perform the simulations for a rocket that we have a Flight object for, but we want to change some parameters of the simulations.

In [ ]:
TestDispersion2 = Dispersion(filename="dispersion_analysis_outputs/disp_class_example2")


In [ ]:
aerodynamic_surfaces = Calisto.aerodynamicSurfaces
nose, fins, tail = aerodynamic_surfaces


In [ ]:
dispersion_dictionary2 = {
    # Environment Parameters
    "railLength": (Env.railLength, 0.001),
    "date": [Env.date],
    "datum": ["WSG84"],
    "elevation": (Env.elevation, 10),
    "gravity": (Env.gravity, 0),
    "latitude": (Env.latitude, 0),
    "longitude": (Env.longitude, 0),
    "timeZone": [str(Env.timeZone)],
    # Solid Motor Parameters
    "burnOutTime": (Pro75M1670.burnOutTime, 0.2),
    "grainDensity": (Pro75M1670.grainDensity, 0.1 * Pro75M1670.grainDensity),
    "grainInitialHeight": (Pro75M1670.grainInitialHeight, 0.001),
    "grainInitialInnerRadius": (Pro75M1670.grainInitialInnerRadius, 0.001),
    "grainNumber": [Pro75M1670.grainNumber],
    "grainOuterRadius": (Pro75M1670.grainOuterRadius, 0.001),
    "grainSeparation": (Pro75M1670.grainSeparation, 0.001),
    "nozzleRadius": (Pro75M1670.nozzleRadius, 0.001),
    "throatRadius": (Pro75M1670.throatRadius, 0.001),
    "thrustSource": [Pro75M1670.thrustSource],
    "totalImpulse": (Pro75M1670.totalImpulse, 0.033 * Pro75M1670.totalImpulse),
    "grainsCenterOfMassPosition": (Pro75M1670.grainsCenterOfMassPosition, 0.001),
    # Rocket Parameters
    "mass": (Calisto.mass, 0.100),
    "radius": (Calisto.radius, 0.001),
    "inertiaI": (Calisto.inertiaI, Calisto.inertiaI * 0.1),
    "inertiaZ": (Calisto.inertiaZ, Calisto.inertiaZ * 0.1),
    "powerOffDrag":["../../../data/calisto/powerOffDragCurve.csv"],
    "powerOnDrag":["../../../data/calisto/powerOnDragCurve.csv"],
    "powerOffDragFactor": (1, 0.033),
    "powerOnDragFactor": (1, 0.033),
    "motor_position": (Calisto.motorPosition, 0.001),
    "nose_name_kind": [nose[0].kind],
    "nose_name_length": (nose[0].length, 0.001),
    "nose_name_position": (nose[1], 0.001),
    "finSet_name_n": [fins[0].n],
    "finSet_name_rootChord": (fins[0].rootChord, 0.001),
    "finSet_name_tipChord": (fins[0].tipChord, 0.001),
    "finSet_name_span": (fins[0].span, 0.001),
    "finSet_name_position": (fins[1], 0.001),
    "finSet_name_airfoil": [fins[0].airfoil],
    "tail_name_topRadius": (tail[0].topRadius, 0.001),
    "tail_name_bottomRadius": (tail[0].bottomRadius, 0.001),
    "tail_name_length": (tail[0].length, 0.001),
    "tail_name_position": (tail[1], 0.001),
    "parachute_Main_CdS": (10,2),
    "parachute_Main_trigger": mainTrigger,
    "parachute_Main_samplingRate": (105,0),
    "parachute_Main_lag": (1.5,0),
    "parachute_Main_noise": [(0,8.3,0.5)],
    "parachute_Drogue_CdS": (1,0.3),
    "parachute_Drogue_trigger": drogueTrigger,
    "parachute_Drogue_samplingRate": (105,0),
    "parachute_Drogue_lag": (1.5,0),
    "parachute_Drogue_noise": [(0,8.3,0.5)],
    # Flight Parameters
    "inclination": (85,1),
    "heading": (90,2),
}

In [ ]:
TestDispersion2.run_dispersion(
    number_of_simulations=50,
    dispersion_dictionary=dispersion_dictionary2,
)


In [ ]:
TestDispersion2.import_results()


And finally, we can export the ellipses of the results to a .kml file so it can be opened on Google Earth

In [ ]:
TestDispersion2.print_results()


In [ ]:
TestDispersion2.allInfo()
